**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

**Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.**

In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st 
import os

pth1 = 'users_behavior.csv'
pth2 = './datasets/users_behavior.csv'
pth3 = '/datasets/users_behavior.csv'
pth4 = './users_behavior.csv'
pth5= '/content/drive/MyDrive/Colab Notebooks/datasets/users_behavior.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
elif os.path.exists(pth3):
    df = pd.read_csv(pth3)
elif os.path.exists(pth4):
    df = pd.read_csv(pth4)
elif os.path.exists(pth5):
    df = pd.read_csv(pth5)
else:
    print("Проверьте правильность пути к датасету")

display(df.head(5))
display(df.describe())
display(df.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Хороший первичный анализ данных

# Разделите исходные данные на обучающую, валидационную и тестовую выборки.


* Обучающая выборка используется для обучения модели,
* Валидационная - для ее настройки,
* Тестовая - для окончательной оценки производительности модели.

Спрятанной тестовой выборки нет. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1. 

In [104]:
df['calls'] = df['calls'].astype('int64') # корректируем тип данных в столбце calls
from sklearn.model_selection import train_test_split

# разбиение на обучающую выборку и валидационные данные
df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)
display('Размер тренировочной выборки:', df_train.shape)
display('Размер вылидационный выборки:', df_valid.shape)
# разбиение обучающих данных на обучающую и тестовую тестовую выборки
df_train, df_test = train_test_split(df_train, test_size=0.25, random_state=12345)
display()
display('Размер тестовой выборки:', df_test.shape)


'Размер тренировочной выборки:'

(2410, 5)

'Размер вылидационный выборки:'

(804, 5)

'Размер тестовой выборки:'

(603, 5)

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Стоит еще выводить размеры выборок для проверки, все ли прошло успешно. Например, с помощью shape

# Исследуйте качество разных моделей, меняя гиперпараметры.


Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 
* Гиперпараметры проверяются на валидационной выборке.




In [105]:
features_train = df_train.drop('is_ultra', axis=1) # признаки для обучения обучающей выборки
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1) # признаки для обучения валидационной выборки
target_valid = df_valid['is_ultra']


features_test = df_test.drop('is_ultra', axis=1) # признаки для обучения тестовой выборки
target_test = df_test['is_ultra']

## Модель дерева решений


In [106]:
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier 

best_model_dtc = None
best_result_dtc = 0

for splitter in ["best", "random"]:
    for criterion in ["gini", "entropy"]:
        for depth in range(1, 16):
            model = DecisionTreeClassifier(criterion=criterion, splitter=splitter, random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
            model.fit(features_train, target_train) # обучите модель
            predictions = model.predict(features_valid)
            result = accuracy_score(target_valid, predictions)
            if result > best_result_dtc:
                best_model_dtc = model
                best_result_dtc = result
                best_depth_dtc = depth
                best_criterion = criterion
                best_splitter = splitter
print("Наилучшее качество модели дерева решений:", best_result_dtc, "При глубине:", best_depth_dtc, "При параметре criterion:", best_criterion, "При параметре splitter:", best_splitter)

Наилучшее качество модели дерева решений: 0.8022388059701493 При глубине: 7 При параметре criterion: entropy При параметре splitter: best


## Модель случайного леса

In [107]:
from sklearn.ensemble import RandomForestClassifier
best_model_rfc = None
best_result_rfc = 0
best_est_rfc = 0

for criterion in ["gini", "entropy"]:
    for est in range(1, 16):
        for depth in range(1, 16):
            model = RandomForestClassifier(criterion = criterion, max_depth=depth, random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
            model.fit(features, target) # обучите модель на тренировочной выборке
            predictions = model.predict(features_valid)
            result = accuracy_score(predictions, target_valid) # посчитайте качество модели на валидационной выборке 
            if result > best_result_rfc:
                best_model_rfc = model
                best_result_rfc = result
                best_depth_rfc = depth
                best_criterion_rfc = criterion
                best_est_rfc = est
print("Наилучшее качество модели случайного леса:", best_result_rfc, "При глубине:", best_depth_rfc, "При параметре criterion:", best_criterion_rfc, "При параметре n_estimators:", best_est_rfc)

Наилучшее качество модели случайного леса: 0.8121890547263682 При глубине: 10 При параметре criterion: gini При параметре n_estimators: 14


## Модель логистической регрессии

In [108]:
from sklearn.linear_model import LogisticRegression
best_model_lr = None
best_result_lr = 0
best_solver_lr = []
best_iter_lr = 0
for solver in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']:
    for i in [10,50,100,500,1000,10000]:
        model = LogisticRegression(random_state=12345, solver=solver, max_iter=i)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)# посчитайте качество модели на валидационной выборке 
        if result > best_result_lr:
            best_model_lr = model
            best_result_lr = result
            best_solver_lr = solver
            best_iter_lr = i
print("Наилучшее качество модели случайного леса:", best_result_lr, "При параметре solver:", best_solver_lr, "При параметре max_iter:", best_iter_lr) # сохраните наилучшую модель

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.9/site-packages/sklearn/utils/optimize.py:202: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search

Наилучшее качество модели случайного леса: 0.7611940298507462 При параметре solver: newton-cg При параметре max_iter: 50


# Сравнение качества моделей на валидационной выборке

In [109]:
# Дерево решений
result_dtc = best_model_dtc.score(features_valid, target_valid)
print("Качество модели дерева решений:", result_dtc)
# Случайный лес
result_rfc = best_model_rfc.score(features_valid, target_valid)
print("Качество модели случайного леса:", result_rfc)
# Логистическая ререссия
result_lr = best_model_lr.score(features_valid, target_valid)
print("Качество модели логистической регрессии:", result_lr)

Качество модели дерева решений: 0.8022388059701493
Качество модели случайного леса: 0.8121890547263682
Качество модели логистической регрессии: 0.7611940298507462


 Предварительно:

1. Самое высокое качество у модели **случайного леса**

2. На втором месте — модель **дерева решений**.

3. Самое низкое качество предсказания у модели **логистической регрессии**.

# Окончательная проверки качества моделей на тестовой выборке


In [110]:
# Дерево решений
result_dtc = best_model_dtc.score(features_test, target_test)
print("Качество модели дерева решений:", result_dtc)
# Случайный лес
result_rfc = best_model_rfc.score(features_test, target_test)
print("Качество модели случайного леса:", result_rfc)
# Логистическая ререссия
result_lr = best_model_lr.score(features_test, target_test) 
print("Качество модели логистической регрессии:", result_lr)

Качество модели дерева решений: 0.7893864013266998
Качество модели случайного леса: 0.8109452736318408
Качество модели логистической регрессии: 0.7412935323383084


 

1. Самое высокое качество у **случайного леса**

2. На втором месте — **дерева решений**.

3. Самое низкое качество предсказания у модели **логистической регрессии**. 

Как видно из проверки на тестовой выборке, **модель случайного леса** показала себя лучше всего во всех наблюдениях, а модель дерева решений сильно снизила свое качество( с 0.8 до 0.78) на тестовой выборке. Самое низкое качество сохраняеется у модели логистической регрессии

In [115]:
print('Баланс классов тренировочной выборки:', target_train.value_counts())

print('Баланс классов валидационной выборки:', target_valid.value_counts())


print('Баланс классов тестовой выборки:', target_test.value_counts())


Баланс классов тренировочной выборки: 0    1255
1     552
Name: is_ultra, dtype: int64
Баланс классов валидационной выборки: 0    563
1    241
Name: is_ultra, dtype: int64
Баланс классов тестовой выборки: 0    411
1    192
Name: is_ultra, dtype: int64


# Общий вывод

Исходя из проделанной работы над гиперпараметрами трёх моделей, наилучший подбор тарифа обеспечит **модель случайного леса при глубине: 10, параметре criterion: gini, параметре n_estimators: 14**. 